# Week 3 
# Day2 :Aug 23
## 1. Write down a few functions (linear, quadratic, etc.) and compute their partial derivatives by hand. You can assume the input to be scalar values
## 2. Write Python code to compute the gradients for the same functions with Numpy and verify if the results match with the ones you computed manually 

In [2]:
import numpy as np

In [18]:
def forward_propagation(x):

    A =  x ** 2
    B = 2*x
    C = A + B + 1
 
    
    return C

In [19]:
x = 3
J = forward_propagation(x)
print ("J = " + str(J))

J = 16


In [15]:
def backward_propagation(x):

    grad = 2 * x + 2
    
    return grad

In [23]:
x = 3
grad = backward_propagation(x)
print ("grad= " + str(grad))

grad= 8


In [30]:
def gradient_check(x, epsilon = 1e-7):
    J_plus = forward_propagation(x+epsilon)      
    J_minus = forward_propagation(x-epsilon)     
    gradapprox = (J_plus - J_minus) / (2. * epsilon) 
    grad = backward_propagation(x)
    print(grad,gradapprox)
    numerator = np.linalg.norm(grad - gradapprox)                    
    denominator = np.linalg.norm(grad) + np.linalg.norm(gradapprox)  
    difference = numerator / denominator                             

    if difference < 1e-7:
        print ("The gradient is correct!")
    else:
        print ("The gradient is wrong!")
    
    return difference

In [26]:
x = 2
difference = gradient_check(x)
print("difference = " + str(difference))

6
The gradient is correct!
difference = 8.182894368803594e-10


In [35]:
import numpy 
x = numpy.array([2.,3.])
difference = gradient_check(x)
print("difference = " + str(difference))

[6. 8.] [5.99999999 7.99999999]
The gradient is correct!
difference = 8.182894368803594e-10


In [32]:
import numpy 
x = numpy.array([2.,3.])
difference = gradient_check(x)
print("difference = " + str(difference))

[6. 8.] [5.99999999 7.99999999]
The gradient is correct!
difference = 8.182894368803594e-10


## 3. Write Python code to compute the gradients for the same functions with PyTorch tensors without using PyTorch's autograd. Verify if the results match with the above ones

In [38]:
import torch
x = torch.tensor(2.0)
grad=backward_propagation(x)
print(grad)

tensor(6.)


In [39]:
import torch
x = torch.tensor(([2., 3.],[1.,3.]))
grad=backward_propagation(x)
print(grad)

tensor([[6., 8.],
        [4., 8.]])


## 4. Use PyTorch autograd to compute the gradients and verify if the results match with the ones from above

In [40]:
import torch
x = torch.tensor(2.0, requires_grad = True)
y = x**2 + 2*x + 1
y.backward()
dx = x.grad
print("x.grad :", int(dx))

x.grad : 6


## 5. Do the same for vector/tensor inputs. Note down the shape of the results at each step and provide proper reasoning for the same

For vector values:

In [ ]:
import torch
x = torch.tensor([2., 3.], requires_grad=True)
y = x**2 + 2*x + 1
external_grad = torch.ones_like(x)
y.backward(gradient=external_grad)
print(x.grad)
print(x.grad.shape)

tensor([6., 8.])
torch.Size([2])


For tensor values:

In [ ]:
import torch

x = torch.tensor(([2., 3.],[1.,3.]), requires_grad=True)
y = x**2 + 2*x + 1
external_grad = torch.ones_like(x)
y.backward(gradient=external_grad)
print(x.grad)
print(x.grad.shape)

tensor([[6., 8.],
        [4., 8.]])
torch.Size([2, 2])


## 6. The CS231n lecture has a slide on "Patterns in backward flow" explaining about interesting properties of the gradients of specific functions. Demonstrate those properties with simple code snippets.

Add gate:

gradient distributor - The add gate always takes the gradient on its output and distributes it equally to all of its inputs, regardless of what their values were during the forward pass.

max gate:  
gradient router - the max gate distributes the gradient (unchanged) to exactly one of its inputs (the input that had the highest value during the forward pass).

mul gate: 

gradient switcher -  Its gradients are the switched input values  multiplied by the gradient on its output during the chain rule.

In [1]:
class MultiplyGate(object):
  def forward(self,x,y):
    z=x*y
    self.x=x
    self.y=y
    return z
  def backward(self,dz):
    dx=self.y*dz
    dy=self.x*dz
    return [dx,dy]

class AddGate(object):
  def forward(self,x,y):
    z=x+y
    self.x=x
    self.y=y
    return z
  def backward(self,dz):
    dx=dz
    dy=dz
    return [dx,dy]

class MaxGate(object):
  def forward(self,x,y):
    z=max(x,y)
    self.x=x
    self.y=y
    return z
  def backward(self,dz):
    if x>y:
      dx=dz
      dy=0.0
    else:
      dx=0.0
      dy=dz
    return [dx,dy]

In [3]:
x=-2.
y=5.
a=-4.
b=3.
add=AddGate()
multiply=MultiplyGate()
maximum=MaxGate()
c=add.forward(x,y)
print("c:",c)
e =maximum.forward(a,b)
print("e:",e)
d=multiply.forward(c,e)
print("d:",d)

c: 3.0
e: 3.0
d: 9.0
